In [132]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv

In [133]:
URL = 'https://auto.drom.ru/geely/preface/'
HEADERS = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36', 'accept':'*/*'}
FILE = 'cars.csv'

In [134]:
def get_html(url,params=None):
    r = requests.get(url, headers=HEADERS,params=params)
    return r

In [135]:
def get_page_count(html):
    soup = bs(html,'html.parser')
    pagination = soup.find_all('a', class_='_1j1e08n0')
    if pagination:
        return int(pagination[-2].get_text())
    else:
        return 1

In [136]:
def all_links(): # Получаеи ссылки всех страниц для парсинга и записываем их в массив, для дальнейшего взаимодействия
    links = list()
    for i in range(1, 501):
        links.append(URL + 'page'+str(i))
    return links

In [137]:
all_links()

['https://auto.drom.ru/geely/preface/page1',
 'https://auto.drom.ru/geely/preface/page2',
 'https://auto.drom.ru/geely/preface/page3',
 'https://auto.drom.ru/geely/preface/page4',
 'https://auto.drom.ru/geely/preface/page5',
 'https://auto.drom.ru/geely/preface/page6',
 'https://auto.drom.ru/geely/preface/page7',
 'https://auto.drom.ru/geely/preface/page8',
 'https://auto.drom.ru/geely/preface/page9',
 'https://auto.drom.ru/geely/preface/page10',
 'https://auto.drom.ru/geely/preface/page11',
 'https://auto.drom.ru/geely/preface/page12',
 'https://auto.drom.ru/geely/preface/page13',
 'https://auto.drom.ru/geely/preface/page14',
 'https://auto.drom.ru/geely/preface/page15',
 'https://auto.drom.ru/geely/preface/page16',
 'https://auto.drom.ru/geely/preface/page17',
 'https://auto.drom.ru/geely/preface/page18',
 'https://auto.drom.ru/geely/preface/page19',
 'https://auto.drom.ru/geely/preface/page20',
 'https://auto.drom.ru/geely/preface/page21',
 'https://auto.drom.ru/geely/preface/page22

In [138]:
def get_content(html):
    soup = bs(html,'html.parser')
    items = soup.find_all('div', class_='css-1f68fiz ea1vuk60')
    
    cars = []

    for item in items:
        cars.append({
            'title':item.find('h3',class_='efwtv890').get_text(strip=True),
            'link':item.find('a',class_='g6gv8w4 g6gv8w8 _1ioeqy90').get('href'),
            'parameters':item.find('div', class_='css-1fe6w6s e162wx9x0').get_text(),
            'city':item.find('span', class_='css-1488ad e162wx9x0').get_text(),
            'price': item.find('span', class_='css-46itwz e162wx9x0').get_text(strip=True)
        })
    return cars

In [139]:
def save_file(items, path):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['name','link','parameters','city','price_rub'])
        for item in items:
            writer.writerow([item['title'],item['link'],item['parameters'],item['city'],item['price']])

In [140]:
def parse():
    html = get_html(URL)
    if html.status_code == 200:
        cars = []
        pages_count = get_page_count(html.text)
        for page in range(1, pages_count+1):
            print(f'Парсинг страницы {page} из {pages_count}....')
            html = get_html(URL, params={'page': page})
            cars.extend(get_content(html.text))
            save_file(cars, FILE)
        print(f'Получены объявления {len(cars)} автомобилей')
    else:
        print('error')

In [141]:
parse()

Парсинг страницы 1 из 10....
Парсинг страницы 2 из 10....
Парсинг страницы 3 из 10....
Парсинг страницы 4 из 10....
Парсинг страницы 5 из 10....
Парсинг страницы 6 из 10....
Парсинг страницы 7 из 10....
Парсинг страницы 8 из 10....
Парсинг страницы 9 из 10....
Парсинг страницы 10 из 10....
Получены объявления 200 автомобилей


In [142]:
# проработать возможность парсинга более 10 страниц для выбранной ссылки / 